# SVM

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
import librosa
import librosa.display
import os
import pickle

### Loading Train and validation data of gunshots and background

In [0]:
gun_train=np.zeros((5594,880))
gun_test=np.zeros((2400,880))
back_train=np.zeros((5594,880))
back_test=np.zeros((2400,880))
ind=0
gun_train_s="E:\\All Data\study\\MS\\2\\machine learning\\Project\\MIVIA_dataset\\MIVIA_DB4_dist\\training\\train_gunshots"
gun_test_s="E:\\All Data\study\\MS\\2\\machine learning\\Project\\MIVIA_dataset\\MIVIA_DB4_dist\\training\\train_gunshots"
back_train_s="E:\\All Data\\study\\MS\\2\\machine learning\\Project\\background_dataset\\train"
back_test_s="E:\\All Data\\study\\MS\\2\\machine learning\\Project\\background_dataset\\test"
path=gun_train_s
hop_length=512
for i in os.listdir(path):
    y, sr = librosa.load(path+"\\"+i,duration=1)
    S = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=20)
    S=np.reshape(S,np.product(S.shape))
    gun_train[ind,:]=S
    ind=ind+1
    if ind>=5594:
        break
print "done loading gun_train"
path=gun_test_s
ind=0
for i in os.listdir(path):
    y, sr = librosa.load(path+"\\"+i,duration=1)
    S = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=20)
#    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=20,fmax=8000)
#    S=librosa.feature.mfcc(S=librosa.power_to_db(S))
    S=np.reshape(S,np.product(S.shape))
    gun_test[ind,:]=S
    ind=ind+1
    if ind>=2400:
        break
print "done loading gun_test"
path=back_train_s
ind=0
for i in os.listdir(path):
    y, sr = librosa.load(path+"\\"+i,duration=1)
    S = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=20)
#    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=20,fmax=8000)
#    S=librosa.feature.mfcc(S=librosa.power_to_db(S))
    S=np.reshape(S,np.product(S.shape))
    if S.shape[0]<880:    
        back_train[ind,0:S.shape[0]]=S[0:S.shape[0]]
    else:
        back_train[ind,:]=S[0:880]
    ind=ind+1
    if ind>=5594:
        break
print "done loading back_train"
path=back_test_s
ind=0
for i in os.listdir(path):
    y, sr = librosa.load(path+"\\"+i,duration=1)
    S = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=20)
#    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=20,fmax=8000)
#    S=librosa.feature.mfcc(S=librosa.power_to_db(S))
    S=np.reshape(S,np.product(S.shape))
    if S.shape[0]<880:    
        back_test[ind,:]=np.concatenate((S[0:860],S[0:20]))
    else:
        back_test[ind,:]=S[0:880]
    ind=ind+1
    if ind>=2400:
        break
print "done loading back_test"


done loading gun_train
done loading gun_test
done loading back_train
done loading back_test


#### Setting data for training and validation

In [0]:
train_X4=gun_train
test_X4=gun_test
train_y4=np.zeros((len(gun_train)))
test_y4=np.zeros((len(gun_test)))

train_X9=back_train
test_X9=back_test
train_y9=np.ones((len(back_train)))
test_y9=np.ones((len(back_test)))

train_set=np.concatenate((train_X4,train_X9),axis=0)
test_set=np.concatenate((test_X4,test_X9),axis=0)


train_y=np.concatenate((train_y4,train_y9),axis=0)
test_y=np.concatenate((test_y4,test_y9),axis=0)

X=np.concatenate((train_set,test_set))
y=np.concatenate((train_y,test_y))


print(train_set.shape,test_set.shape,train_y.shape,test_y.shape)
print(len(train_X4),len(test_X4),len(train_X9),len(test_X9))



((11188L, 880L), (4800L, 880L), (11188L,), (4800L,))
(5594, 2400, 5594, 2400)


### Training with Inhomogenous polynomial kernel (u*v+1)^p  p=1,2

In [0]:
'''
import cPickle
save_training = open('E:\\All Data\study\\MS\\2\\machine learning\\Project\\save_training.pickle','wb')
cPickle.dump(cl,save_training)  # SAVE TRAINED CLASSIFIER
save_training.close()
load_training = open(''E:\\All Data\study\\MS\\2\\machine learning\\Project\\save_training.pickle'','rb')
new_cl = cPickle.load(load_training) # LOAD TRAINED CLASSIFIER
'''

"\nimport cPickle\nsave_training = open('E:\\All Data\\study\\MS\\2\\machine learning\\Project\\save_training.pickle','wb')\ncPickle.dump(cl,save_training)  # SAVE TRAINED CLASSIFIER\nsave_training.close()\nload_training = open(''E:\\All Data\\study\\MS\\2\\machine learning\\Project\\save_training.pickle'','rb')\nnew_cl = cPickle.load(load_training) # LOAD TRAINED CLASSIFIER\n"

### degree=p=1

In [0]:
error=[]
c=[]
for i in range(20):
    c.append(1e-10*10**i)
    clf1= svm.SVC(C=c[i],kernel='poly',degree=1)
    clf1.fit(train_set,train_y)
    Pe = 1 - clf1.score(test_set,test_y)
    error.append(Pe)

for i in range(len(error)):
    print("for C=%f test_error=%f"%(c[i],error[i]))
minimum_error=np.argmin(error)
print ("\n\n for C=%f got minimum test_error=%f"%(c[minimum_error],error[minimum_error]))
print ("C=%f was selected to retrain the classifier"%(c[minimum_error]))

#retraining on full dataset for 
'''
train_X=np.concatenate((X4,X9),axis=0)
train_yf=np.concatenate((y4,y9),axis=0)
'''
clf1= svm.SVC(C=c[minimum_error],kernel='poly',degree=1)
clf1.fit(X,y)
Pe = 1 - clf1.score(test_set,test_y)

print ("\n\n for C=%f got minimum test_error=%f after retraining"%(c[minimum_error],Pe))

print "saving trained model"
save_training = open('E:\\All Data\study\\MS\\2\\machine learning\\Project\\save_training_1.pickle','wb')
pickle.dump(clf1, save_training, protocol=2)
#pickle.dumps(clf1,save_training)  # SAVE TRAINED CLASSIFIER
save_training.close()


for C=0.000000 test_error=0.187292
for C=0.000000 test_error=0.187292
for C=0.000000 test_error=0.187292
for C=0.000000 test_error=0.187292
for C=0.000001 test_error=0.012500
for C=0.000010 test_error=0.001458
for C=0.000100 test_error=0.001042
for C=0.001000 test_error=0.000417
for C=0.010000 test_error=0.000417
for C=0.100000 test_error=0.000417
for C=1.000000 test_error=0.000417
for C=10.000000 test_error=0.000417
for C=100.000000 test_error=0.000417
for C=1000.000000 test_error=0.000417
for C=10000.000000 test_error=0.000417
for C=100000.000000 test_error=0.000417
for C=1000000.000000 test_error=0.000417
for C=10000000.000000 test_error=0.000417
for C=100000000.000000 test_error=0.000417
for C=1000000000.000000 test_error=0.000417


 for C=0.001000 got minimum test_error=0.000417
C=0.001000 was selected to retrain the classifier


 for C=0.001000 got minimum test_error=0.000000 after retraining
saving trained model


In [0]:
print "loading trained model"
load_training = open("E:\\All Data\study\\MS\\2\\machine learning\\Project\\save_training_1.pickle",'rb')
new_clf=pickle.load(load_training) # LOAD TRAINED CLASSIFIER

#getting support vectors number and indices
support=new_clf.n_support_
print ("number of support_vectors=",sum(support))


loading trained model
('number of support_vectors=', 68)


### degree=p=2

In [0]:
error=[]
c=[]
for i in range(20):
    c.append(1e-10*10**i)
    clf2= svm.SVC(C=c[i],kernel='poly',degree=2)
    clf2.fit(train_set,train_y)
    Pe = 1 - clf2.score(test_set,test_y)
    error.append(Pe)

for i in range(len(error)):
    print("for C=%f test_error=%f"%(c[i],error[i]))
minimum_error=np.argmin(error)
print ("\n\n for C=%f got minimum test_error=%f"%(c[minimum_error],error[minimum_error]))
print ("C=%f was selected to retrain the classifier"%(c[minimum_error]))


#retraining on full dataset for 
clf2= svm.SVC(C=c[minimum_error],kernel='poly',degree=1)
clf2.fit(X,y)
Pe = 1 - clf2.score(test_set,test_y)

print ("\n\n for C=%f got minimum test_error=%f after retraining"%(c[minimum_error],Pe))

print "saving trained model"
save_training = open('E:\\All Data\study\\MS\\2\\machine learning\\Project\\save_training_2.pickle','wb')
pickle.dump(clf2,save_training,protocol=2)  # SAVE TRAINED CLASSIFIER
save_training.close()
print "loading trained model"
load_training = open("E:\\All Data\study\\MS\\2\\machine learning\\Project\\save_training_2.pickle",'rb')
new_clf = pickle.load(load_training) # LOAD TRAINED CLASSIFIER
#getting support vectors number
support=new_clf.n_support_
print("number of support vectors=%d"%sum(support))

for C=0.000000 test_error=0.017917
for C=0.000000 test_error=0.004792
for C=0.000000 test_error=0.001875
for C=0.000000 test_error=0.000833
for C=0.000001 test_error=0.000417
for C=0.000010 test_error=0.000208
for C=0.000100 test_error=0.000208
for C=0.001000 test_error=0.000208
for C=0.010000 test_error=0.000208
for C=0.100000 test_error=0.000208
for C=1.000000 test_error=0.000208
for C=10.000000 test_error=0.000208
for C=100.000000 test_error=0.000208
for C=1000.000000 test_error=0.000208
for C=10000.000000 test_error=0.000208
for C=100000.000000 test_error=0.000208
for C=1000000.000000 test_error=0.000208
for C=10000000.000000 test_error=0.000208
for C=100000000.000000 test_error=0.000208
for C=1000000000.000000 test_error=0.000208


 for C=0.000010 got minimum test_error=0.000208
C=0.000010 was selected to retrain the classifier


 for C=0.000010 got minimum test_error=0.000000 after retraining
saving trained model
loading trained model
number of support vectors=1647
